let's try to make a better tmatrix

In [944]:
from pylab import mpl, plt
plt.style.use(plt.style.available[11])
mpl.rcParams['font.family'] = 'serif'
%matplotlib inline

In [945]:
from tvDatafeed import TvDatafeed, Interval
username = ''
password = ''

tvl = TvDatafeed(username, password)

ERROR:tvDatafeed.main:error while signin


In [946]:
ticker = 'HDFCBANK'
data=tvl.get_hist(ticker, 'NSE', interval=Interval.in_1_hour, n_bars=300000, fut_contract=None, extended_session=False)

In [947]:
import numpy as np

In [948]:
data['open_close_returns'] = np.log(data['close']) - np.log(data['open'])

In [949]:
data['open_close_returns_adjusted'] = (data['open_close_returns'])*100

In [950]:
data['open_close_returns_adjusted']

datetime
2022-01-03 09:15:00    1.136243
2022-01-03 10:15:00    0.076625
2022-01-03 11:15:00    0.286047
2022-01-03 12:15:00    0.437449
2022-01-03 13:15:00    0.194916
                         ...   
2025-08-14 11:15:00    0.195995
2025-08-14 12:15:00    0.080289
2025-08-14 13:15:00   -0.050173
2025-08-14 14:15:00   -0.070299
2025-08-14 15:15:00   -0.085425
Name: open_close_returns_adjusted, Length: 6269, dtype: float64

In [951]:
data['open_close_returns_adjusted'].quantile(0.25)

np.float64(-0.18770715542224536)

In [952]:
for index in data.index:
    if data.loc[index, 'open_close_returns_adjusted'] < data['open_close_returns_adjusted'].quantile(0.33):
        data.loc[index, 'today_state'] = -1
    elif data.loc[index, 'open_close_returns_adjusted'] > data['open_close_returns_adjusted'].quantile(0.66):
        data.loc[index, 'today_state'] = 1
    else:
        data.loc[index, 'today_state'] = 0

In [953]:
N = 3
state_cols = []
for i in range(1, N+1):
    data[f'{i}_state'] = data['today_state'].shift(i)
    state_cols.append(f'{i}_state')
data = data.dropna(axis=0)

In [954]:
for index in data.index:
    data.loc[index, 'state'] = str([int(data.loc[index, col]) for col in state_cols])

In [955]:
data['today_state'] = data['today_state'].astype(int)

In [956]:
data = data[['today_state', 'state']]

In [957]:
data

,today_state,state
datetime,,
2022-01-03 12:15:00,1,"[1, 0, 1]"
2022-01-03 13:15:00,1,"[1, 1, 0]"
2022-01-03 14:15:00,1,"[1, 1, 1]"
2022-01-03 15:15:00,0,"[1, 1, 1]"
2022-01-04 09:15:00,1,"[0, 1, 1]"
...,...,...
2025-08-14 11:15:00,1,"[0, 1, -1]"
2025-08-14 12:15:00,0,"[1, 0, 1]"
2025-08-14 13:15:00,0,"[0, 1, 0]"


In [958]:
max_prob = 0
for i in {-1, 0, 1}:
    for j in {-1, 0, 1}:
        for k in {-1, 0, 1}:
            # for l in {-1, 0, 1}:
            #     for m in {-1, 0, 1}:
                    temp_list = data[data['state'] == f'[{i}, {j}, {k}]']['today_state'].to_list()
                    probs = max(temp_list.count(-1)/(len(temp_list)), temp_list.count(1)/(len(temp_list)))
                    max_prob = max(probs, max_prob)
print(max_prob)

0.41836734693877553


In [959]:
# for i in {-1, 0, 1}:
#     for j in {-1, 0, 1}:
#         for k in {-1, 0, 1}:
#             for l in {-1, 0, 1}:
#                 plt.figure(figsize=(8,5.5))
#                 data[data['state'] == f'[{i}, {j}, {k}, {l}]']['today_state'].plot(kind='hist', title=f'[{i}, {j}, {k}, {l}]')
#                 plt.show()